In [ ]:
!pip install transformers==4.28.0
!pip install datasets
!pip install pyTelegramBotAPI requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00

In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel,AutoTokenizer, GPT2TokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
def process_csv(file_path):
    df = pd.read_csv("/content/questions.csv", encoding='cp1252')
    #df = pd.read_csv('/content/questions.csv')
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Load and preprocess the dataset
train_dataset = load_dataset("/content/questions.csv", tokenizer)
valid_dataset = load_dataset("/content/valid.csv", tokenizer)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,No log,2.527639
200,No log,2.252000
300,No log,2.033956
400,No log,1.827739
500,2.421400,1.643535
600,2.421400,1.464898
700,2.421400,1.286759
800,2.421400,1.138150
900,2.421400,1.010536
1000,1.549700,0.878520


TrainOutput(global_step=2325, training_loss=1.2994907502205142, metrics={'train_runtime': 1053.7223, 'train_samples_per_second': 17.628, 'train_steps_per_second': 2.206, 'total_flos': 1213374873600000.0, 'train_loss': 1.2994907502205142, 'epoch': 25.0})

In [ ]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_interview_prep_gpt2")


In [ ]:
# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_interview_prep_gpt2")

def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()

    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]

    return answer


# Ask questions using the fine-tuned model
question = " Describe the concept of polymorphism in OOP?"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

question = "What is RDBMS"
answer = ask_question(question, fine_tuned_model, tokenizer)
print(f"Question: {question}\nAnswer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question:  Describe the concept of polymorphism in OOP?
Answer: Polymorphism in OOP allows classes to inherit properties and behaviors from parent classes. It enables code flexibility by allowing the creation of specialized classes that extend the functionality of parent classes by modifying the codebase. Polymorphic inheritance allows for code abstraction and extension by allowing classes to override or extend the parent classes in ways that extend their functionality.
Question: What is RDBMS
Answer: RDBms stands for Relational Database Management System. It stores the related table records in a database, and can be referred or queried to access the related tables in the database. RDBMs are relational database management systems that provide a centralized and organized way to manage and retrieve data from a database.


In [ ]:
import telebot
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_interview_prep_gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Create the Telegram bot
bot_token = "6027166289:AAGqlei_xtRTNj9acFffjrvtlcKf3s7w54s"
bot = telebot.TeleBot(bot_token)

@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.reply_to(message, "Hello! I am your AI-Quest. Feel free to ask me any questions!")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    question = message.text
    answer = ask_question(question)
    if answer:
        bot.reply_to(message, answer)
    else:
        bot.reply_to(message, "I'm sorry, but I couldn't find a solution to your question. Please try asking a different question.")

def ask_question(question):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = fine_tuned_model.generate(
        inputs,
        max_length=128,
        num_return_sequences=1,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()
    answer = answer.split("\n")[0]

    return answer if answer else None

bot.polling()


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [ ]:
!pip install Ipython.display

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement Ipython.display (from versions: none)
ERROR: No matching distribution found for Ipython.display


In [ ]:
from IPython.display import display, HTML

# Define the HTML code
html_code = """
<!DOCTYPE html>
<html>
<head>
    <style>
        /* CSS styles */
    </style>
</head>
<body>
    <div>
        <h1>Welcome to AI-Quest</h1>
        <p>Ask any question, and AI-Quest will provide you with the answer.</p>
        <input type="text" id="question-input" placeholder="Enter your question">
        <button id="ask-button">Ask</button>
        <p id="answer"></p>
    </div>
    <script>
        // JavaScript code
        document.getElementById("ask-button").addEventListener("click", function() {
            var question = document.getElementById("question-input").value;
            // Perform actions with the question input
            // Call your AI model or API to get the answer
            // Update the UI with the answer
            document.getElementById("answer").innerText = "Loading...";
            // Example: Simulating a delay and updating the answer
            setTimeout(function() {
                // Call the ask_question function
                var answer = ask_question(question);
                document.getElementById("answer").innerText = "Answer: " + answer;
            }, 2000);
        });

        // Define the ask_question function
        function ask_question(question) {
            // Call the ask_question function from your project code
            var answer = "This is the answer.";
            return answer;
        }
    </script>
</body>
</html>
"""

# Display the HTML interface in the notebook
display(HTML(html_code))

# Your project code starts here
!pip install transformers==4.28.0
!pip install datasets

import pandas as pd
from transformers import GPT2LMHeadModel, AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from datasets import Dataset

def process_csv(file_path):
    df = pd.read_csv(file_path, encoding='cp1252')
    qa_pairs = []

    for index, row in df.iterrows():
        question = row['question']
        answer = row['answer']
        qa_pairs.append(f"Question: {question}\nAnswer: {answer}\n")

    return qa_pairs

def load_dataset(file_path, tokenizer):
    qa_pairs = process_csv(file_path)
    tokenized_dataset = tokenizer(qa_pairs, truncation=True,
                                  padding='max_length', max_length=128,
                                  return_tensors="pt")
    dataset = Dataset.from_dict(tokenized_dataset)
    return dataset

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Load and preprocess the dataset
train_dataset = load_dataset("/content/questions.csv", tokenizer)
valid_dataset = load_dataset("/content/valid.csv", tokenizer)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
